In [4]:
!pip install opencv-python
!pip install scikit-image
import numpy as np
import cv2
import skimage.color as color
import skimage.io as io
import os

from glob import glob
casia_dir = './casia/CASIA1/'
sp_dir = casia_dir+'Sp/'       
mask_dir = './casia/mask/'  
probe_dir = './casia/probe/'   
au_dir = casia_dir+'/Au'       
ext = 'Sp*'
dataType = '.jpg'

if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)

if not os.path.exists(probe_dir):
    os.makedirs(probe_dir)
filenames = glob(os.path.join(sp_dir, ext))   
im_num = len(filenames)
im_id = 0


You should consider upgrading via the 'c:\users\kushagra\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)


You should consider upgrading via the 'c:\users\kushagra\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.
c:\users\kushagra\appdata\local\programs\python\python38-32\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [5]:
def bounding_box(image, mask, row_data):
    box_list=[]

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    print("contours num：", len(contours))
    image_copy=image.copy()
    contours = sorted(contours, key=lambda i: len(i),reverse=True)
    for i in range(0, len(contours)):
        area=cv2.contourArea(contours[i])
        print(area,mask.shape[0]/20*image.shape[1]/20)
        if area>(mask.shape[0]/20*image.shape[1]/20):
            print(i)
            x, y, w, h = cv2.boundingRect(contours[i])
            x1=x
            y1=y
            x2=x+w
            y2=y+h
            box_list.append(str(x1)+'_'+str(y1)+'_'+ str(x2)+'_'+str(y2))
        else:
            continue

    if len(box_list)==0 and len(contours)!=0:
        x, y, w, h = cv2.boundingRect(contours[0])
        x1 = x
        y1 = y
        x2 = x + w
        y2 = y + h
        box_list.append(str(x1) + '_' + str(y1) + '_' + str(x2) + '_' + str(y2))
    if len(contours)==0 and len(box_list)==0 :
        box_list.append('0' + '_' + '0' + '_' + '0' + '_' + '0')
    return box_list

In [6]:
# Generate CASIA mask
test_name=[]
cls='tamper'
for im in filenames:
    
    img_org = io.imread(im)
    img = img_org.copy()
    img_org = cv2.cvtColor(img_org, cv2.COLOR_RGB2BGR)
    base_name = os.path.splitext(os.path.basename(im))[0]
    content = base_name.split("_")
    au_tmp = str(content[4])
    auth_name = 'Au_' + au_tmp[:-4] + '_' + au_tmp[-4:] + '*'#Authentic image name

    auth_img = io.imread(glob(os.path.join(au_dir, auth_name))[0])
    if (img.shape != auth_img.shape):
        au_tmp = str(content[5])
        auth_name = 'Au_' + au_tmp[:-4] + '_' + au_tmp[-4:] + '*'
        auth_img = io.imread(glob(os.path.join(au_Dir, auth_name))[0])
    img = color.rgb2grey(img)
    auth_img = color.rgb2grey(auth_img)
    
    # Generate coarse mask
    mask = np.abs(img - auth_img) * 255
    # Eliminate noise
    ret, mask = cv2.threshold(mask, 15, 255, cv2.THRESH_BINARY)#After testing, the threshold is set to 15
    mask = mask.astype(np.uint8)
    mask = cv2.medianBlur(mask, 5)#Remove noise
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)#Binarization

    # Generate a new mask that contains only large regions
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours, key=lambda i: len(i), reverse=True)
    mask_save = np.zeros_like(mask, dtype=np.uint8)
    c_max = []
    for i in range(len(contours)):
        cnt = contours[i]
        area = cv2.contourArea(cnt)

        if (area > (mask.shape[0] / 20 * mask.shape[1] / 20) and area < (mask.shape[0] * mask.shape[1] * 0.7)):
            c_max.append(cnt)
    cv2.drawContours(mask_save, c_max, -1, (255), thickness=-1)
    if len(c_max) > 0:
        box_list=[]
        for cnt in c_max:
            x, y, w, h = cv2.boundingRect(contours[i])
            x1=x
            y1=y
            x2=x+w
            y2=y+h
            box_list.append(str(x1)+'_'+str(y1)+'_'+ str(x2)+'_'+str(y2))
            content=str(base_name).split('_')
            print(content)
            save_name='Tp' +'_'+str(content[4])+'_'+str(content[5])+'_'+str(content[3])+str(content[6])+'_'
            for i in range(0,len(box_list)):
                 save_name=save_name+box_list[i]+'_'+cls+'_'
        cv2.imwrite(probe_dir + save_name.rstrip('_')+ ".png", img_org)
        cv2.imwrite(mask_dir + save_name.rstrip('_') + ".png", mask_save)
        test_name.append(save_name.rstrip('_'))
        print('{:d}/{:d} images     {:s}'.format(im_id+1, im_num, save_name.rstrip('_')))
    else:
        continue
    im_id += 1

<ipython-input-6-28923f92360d>:19: FutureWarning: rgb2grey is deprecated. It will be removed in version 0.19.Please use rgb2gray instead.
  img = color.rgb2grey(img)
<ipython-input-6-28923f92360d>:20: FutureWarning: rgb2grey is deprecated. It will be removed in version 0.19.Please use rgb2gray instead.
  auth_img = color.rgb2grey(auth_img)


['Sp', 'D', 'CND', 'A', 'pla0005', 'pla0023', '0281']
1/921 images     Tp_pla0005_pla0023_A0281_113_87_253_177_tamper
['Sp', 'D', 'CND', 'A', 'sec0056', 'sec0015', '0282']
2/921 images     Tp_sec0056_sec0015_A0282_187_183_384_256_tamper
['Sp', 'D', 'CNN', 'A', 'ani0049', 'ani0084', '0266']
3/921 images     Tp_ani0049_ani0084_A0266_96_190_193_256_tamper
['Sp', 'D', 'CNN', 'A', 'ani0053', 'ani0054', '0267']
4/921 images     Tp_ani0053_ani0054_A0267_283_150_284_151_tamper
['Sp', 'D', 'CNN', 'A', 'art0024', 'ani0032', '0268']
5/921 images     Tp_art0024_ani0032_A0268_168_99_273_228_tamper
['Sp', 'D', 'CNN', 'A', 'cha0025', 'pla0067', '0269']
['Sp', 'D', 'CNN', 'A', 'cha0025', 'pla0067', '0269']
['Sp', 'D', 'CNN', 'A', 'cha0025', 'pla0067', '0269']
6/921 images     Tp_cha0025_pla0067_A0269_145_177_150_181_tamper_69_112_123_224_tamper_234_113_288_224_tamper
['Sp', 'D', 'CNN', 'A', 'nat0014', 'nat0017', '0097']
7/921 images     Tp_nat0014_nat0017_A0097_80_310_81_311_tamper
['Sp', 'D', 'CNN', 

70/921 images     Tp_cha0079_cha0005_A0604_226_0_269_55_tamper
['Sp', 'D', 'NND', 'A', 'cha0090', 'cha0082', '0187']
71/921 images     Tp_cha0090_cha0082_A0187_68_128_90_161_tamper
['Sp', 'D', 'NND', 'A', 'cha0094', 'cha0089', '0605']
72/921 images     Tp_cha0094_cha0089_A0605_0_37_79_256_tamper
['Sp', 'D', 'NND', 'A', 'cha0097', 'arc0005', '0606']
73/921 images     Tp_cha0097_arc0005_A0606_195_21_384_78_tamper
['Sp', 'D', 'NND', 'A', 'nat0015', 'nat0014', '0607']
74/921 images     Tp_nat0015_nat0014_A0607_108_14_368_67_tamper
['Sp', 'D', 'NND', 'A', 'nat0022', 'ani0063', '0188']
75/921 images     Tp_nat0022_ani0063_A0188_339_250_343_254_tamper
['Sp', 'D', 'NND', 'A', 'nat0023', 'nat0013', '0608']
76/921 images     Tp_nat0023_nat0013_A0608_122_87_123_88_tamper
['Sp', 'D', 'NND', 'A', 'nat0031', 'nat0033', '0609']
77/921 images     Tp_nat0031_nat0033_A0609_108_148_109_150_tamper
['Sp', 'D', 'NND', 'A', 'nat0031', 'nat0096', '0610']
78/921 images     Tp_nat0031_nat0096_A0610_253_120_254_

IndexError: list index out of range

In [9]:
# Generate the final testing set catalog file
with open('./casia/test_all_single1.txt', 'w') as f:
    for pic in test_name:
        test_content = pic.split("_")
        if test_content[-1] in cls:
            content2 = [str(i) for i in test_content[4:-1]]
            content3=' '.join(content2)
            f.write('%s %s %s\n' % (pic,content3,'tamper'))